In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import copy
import warnings
warnings.filterwarnings('ignore')

import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig

from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader

from utils.data import load_Ford_A, transform_data, MyDataset
from models.models import LSTM_net

from utils.attacks import attack_procedure
from utils.utils import save_experiment, load_disc_model
from utils.config import get_attack, load_disc_config
from utils.TS2Vec.datautils import load_UCR


In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
initialize(config_path='../config', version_base=None)
cfg = compose(config_name='attack_run_config')
print(OmegaConf.to_yaml(cfg))

dataset: GunPointOldVersusYoung
slice: true
use_disc_check: false
model_folder: checkpoints/${dataset}/TS2Vec/
save_path: results/${dataset}/IFGSM/${attack_type}
model_id_attack: 0
attack_model:
  _target_: models.models.TS2VecClassifier
disc_model:
  _target_: models.models.LSTM_net
  hidden_dim: 50
  n_layers: 1
  output_dim: 1
  dropout: 0.0
disc_path: results/${dataset}/IFGSM/Discriminator_pickle
disc_model_check:
  _target_: models.models.LSTM_net
  hidden_dim: 50
  n_layers: 1
  output_dim: 1
  dropout: 0.0
cuda: cuda:2
attack_type: fgsm_attack
n_iterations: 10
all_eps:
- 0.1
train_mode: true
batch_size: 64
alphas:
- 10
disc_check_params:
  model_id: 0
  model_name: fgsm_attack_eps=0.03_nsteps=10



In [8]:
# load data
if cfg['dataset'] == 'Ford_A':
    X_train, X_test, y_train, y_test = load_Ford_A()
else:
    X_train, y_train, X_test, y_test = load_UCR(cfg['dataset'])

X_train, X_test, y_train, y_test = transform_data(X_train, X_test, y_train, y_test, slice_data=cfg['slice'])

test_loader = DataLoader(
    MyDataset(X_test, y_test), 
    batch_size=cfg['batch_size'] , 
    shuffle=False
    )

criterion = torch.nn.BCELoss()
n_objects = y_test.shape[0]
device= torch.device(cfg['cuda'] if torch.cuda.is_available() else 'cpu')
attack_func = get_attack(cfg['attack_type'])


model = instantiate(cfg.attack_model).to(device)

disc_model = instantiate(cfg.disc_model).to(device)

if cfg['use_disc_check']:
    disc_model_check = instantiate(cfg.disc_model_check).to(device)
    disc_model_check = load_disc_model(
        copy.deepcopy(disc_model_check),
        model_id=cfg['disc_check_params']['model_id'], 
        path=cfg['disc_path'], 
        model_name=cfg['disc_check_params']['model_name'], 
        device=device
        )
    disc_model_check.eval()
else: 
    disc_model_check = None

alphas = [0]
if 'reg' in cfg['attack_type'] or 'disc' in cfg['attack_type']:
    alphas = cfg['alphas']

for alpha in tqdm(alphas):

    attack_params = dict()

    if 'reg' in cfg['attack_type'] :
        attack_params['alpha'] = alpha

    elif 'disc' in cfg['attack_type']:
        attack_params['alpha'] = alpha
        attack_params['disc_models'] = load_disc_config(
            copy.deepcopy(disc_model),
            cfg['disc_path'], 
            device, 
            cfg['list_reg_model_params']
            )  
        
        attack_params['disc_models'] = [model.eval() for model in  attack_params['disc_models']]
    
    model_path = cfg['model_folder'] + f'model_{cfg["model_id_attack"]}_{cfg["dataset"]}.pth'
    model.load_state_dict(copy.deepcopy(torch.load(model_path)))
    model.eval()

100%|██████████| 1/1 [00:00<00:00, 35.50it/s]


In [9]:
X_train.shape

torch.Size([408, 50])

In [10]:
y_true = torch.Tensor()
y_pred = torch.Tensor()

for X, y in test_loader:
    pred = model(X.to(device))
    print(pred.flatten()[:10], y.flatten()[:10])

    y_true = torch.cat([y_true, y])
    y_pred = torch.cat([y_pred, pred.cpu()])
    if 1 in pred.to(int):
        print('1 is detected')
        break
        

tensor([0.2397, 0.2227, 0.2585, 0.2803, 0.3318, 0.2228, 0.2867, 0.3565, 0.2360,
        0.3544], device='cuda:2', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.2531, 0.2863, 0.2862, 0.2658, 0.2865, 0.2841, 0.2719, 0.3055, 0.2765,
        0.2610], device='cuda:2', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.2670, 0.3452, 0.2862, 0.3133, 0.3165, 0.2981, 0.2817, 0.2583, 0.2585,
        0.2569], device='cuda:2', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.6477, 0.7801, 0.7936, 0.7923, 0.6777, 0.7748, 0.8004, 0.6859, 0.6816,
        0.7812], device='cuda:2', grad_fn=<SliceBackward0>) tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0.7988, 0.6751, 0.7901, 0.7987, 0.7822, 0.7997, 0.7926, 0.6846, 0.6364,
        0.7764], device='cuda:2', grad_fn=<SliceBackward0>) tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0.2261, 0.2910, 0.3591, 0.2403, 0.3572, 0.2634,

In [11]:
#accuracy
torch.sum(y_true.flatten() == (y_pred.flatten() > 0.5)) / len(y_true.flatten())

tensor(0.9841)